In [1]:
import numpy as np
import pandas as pd
import spiceypy as spice
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.constants import c
from astropy.time import Time
from astroquery.jplhorizons import Horizons
import rebound
import random

In [2]:
def Furnisher(k):
    '''
    This function is used to load all kernels needed in an operation.
    Comment out kernels not in use and add the ones in use.
    
    Arguments: NA
    Returns: NA
    
    '''
    spice.kclear()
    spice.furnsh('/Users/user/Downloads/naif0008.tls.txt')
    if k == '310+341+435':
            spice.furnsh('/Users/user/Downloads/jup310.bsp')
            spice.furnsh('/Users/user/Downloads/jup341.bsp')
            spice.furnsh('/Users/user/Downloads/de435.bsp')
    elif k == '310+341':
            spice.furnsh('/Users/user/Downloads/jup310.bsp')
            spice.furnsh('/Users/user/Downloads/jup341.bsp')
    elif k == '310+435':
            spice.furnsh('/Users/user/Downloads/de435.bsp')
            spice.furnsh('/Users/user/Downloads/jup310.bsp')
    elif k == '341+435':
            spice.furnsh('/Users/user/Downloads/jup341.bsp')
            spice.furnsh('/Users/user/Downloads/de435.bsp')
    elif k == '310':
            spice.furnsh('/Users/user/Downloads/jup310.bsp')
    elif k == '341':
            spice.furnsh('/Users/user/Downloads/jup341.bsp')
    elif k == '435':
            spice.furnsh('/Users/user/Downloads/de435.bsp')
    pass

In [3]:
def get_spice_function(name,cor,loc):
    """
    This wrapper function automates the creation of objects through the JPL Horizons database. 
    
    Arguments:
    
    name: str
    
    Stipulates the target object in Horizons. The major bodies in the Solar System have an id based on their position.
    Hence '5' refers to Jupiter and '3' to Earth. A single number designator refers to a barycenter and a designator
    such as '599' to the planetary center. For minor bodies in the Solar System, the id_type in the Horizons
    must be changed to "minorbody"
    
    cor: str
    
    Refers to the type of correction that the object has. Available arguments are 'NONE', 'LT','LT+S'
    
    loc: str
    
    Designates the location of observation. Names that start with "g@#" refer to barycenters where the number designates the 
    body that the observer is based at. Hence "g@0" refers to the Solar System barycenter. Also takes Earth location designators.
    Observatories are named after their code. Hence, Pan-Starrs observatory is referred as "f51"

    Returns:
    
    get_target_xyz function
    """    
    def get_target_xyz(t):
        """
        Returns the vectors of the Horizons body at a certain time t.
        
        Arguments:
        
        t: days
        
        Julian date of observation
        
        Returns:
    
        xyz: numpy array
        
        A position vector of the observed object
    
        uvw: numpy array
        
        An instantaneous velocity vector of the observed object
        
        radec: numpy array
        
        The right ascension and declination of the observed object
        """
        
        state,lighttime = spice.spkezr(name,t,'J2000',cor,loc)
        pos,lighttime = spice.spkpos(name,t,'J2000',cor,loc)
        range,ra,dec = spice.recrad(pos) 
        xyz = np.array([state[0],state[1],state[2]])/149597870.7#6.68459e-9
        uvw = np.array([state[3],state[4],state[5]])/149597870.7*24.*3600.#*6.68459e-9
        radec = np.array([ra,dec])
        return xyz,uvw,radec*180/np.pi
    return get_target_xyz

In [4]:
# From 432 (km^3/s^2)
GMEarth = 398600.435420
GMMoon = 27068703.151194

GMMercuryB = 22031.780000
GMVenusB = 324858.592000
GMEarthB = 403503.235502
GMMarsB = 42828.375214
GMJupiterB = 126712764.133446
GMSaturnB = 37940585.200000
GMUranusB = 5794556.465752
GMNeptuneB = 6836527.100580
GMSun = 132712440041.939400 # Just Sun
GMSunterr = GMSun + GMMercuryB + GMVenusB + GMEarthB + GMMarsB

# From 310

GMJupiter = 1.266865341960128E+08
GMIo = 5.959924010272514E+03
GMEuropa = 3.202739815114734E+03
GMGanymede = 9.887819980080976E+03
GMCallisto = 7.179304867611079E+03
GMAmalthea = 1.487604677404272E-01
GM10310 = 1.327132332639000E+11

GMSaturnB310 = 3.794058500000000E+07
GMUranusB310 = 5.794548600000000E+06
GMNeptuneB310 = 6.836527100580000E+06
GMJupiterB310 = 1.267127641334463E+08

In [5]:
MMercuryB = GMMercuryB/GMSun
MVenusB = GMVenusB/GMSun
MEarthB = GMEarthB/GMSun
MEarth = GMEarth/GMSun
MMoon = GMMoon/GMSun
MMarsB = GMMarsB/GMSun
MJupiter = (GMJupiterB-GMIo-GMEuropa-GMGanymede-GMCallisto)/GMSun
MSaturnB = GMSaturnB/GMSun
MUranusB = GMUranusB/GMSun
MNeptuneB = GMNeptuneB/GMSun
MIo = GMIo/GMSun
MEuropa = GMEuropa/GMSun
MGanymede = GMGanymede/GMSun
MCallisto = GMCallisto/GMSun
MAmalthea = GMAmalthea/GMSun

In [6]:
Furnisher("310+341+435")

In [7]:
def get_astroquery_function(name,cor,loc):
    """
    This wrapper function automates the creation of objects through the JPL Horizons database. 
    
    Arguments:
    
    name: str
    
    Stipulates the target object in Horizons. The major bodies in the Solar System have an id based on their position.
    Hence '5' refers to Jupiter and '3' to Earth. A single number designator refers to a barycenter and a designator
    such as '599' to the planetary center. For minor bodies in the Solar System, the id_type in the Horizons
    must be changed to "minorbody"
    
    cor: str
    
    Refers to the type of correction that the object has. Available arguments are "geometric","astrometric" and 
    "apparent"
    
    loc: str
    
    Designates the location of observation. Names that start with "g@#" refer to barycenters where the number designates the 
    body that the observer is based at. Hence "g@0" refers to the Solar System barycenter. Also takes Earth location designators.
    Observatories are named after their code. Hence, Pan-Starrs observatory is referred as "f51"

    Returns:
    
    get_target_xyz function
    """    
    def get_target_xyz(t):
        """
        Returns the vectors of the Horizons body at a certain time t.
        
        Arguments:
        
        t: days
        
        Julian date of observation
        
        Returns:
    
        xyz: numpy array
        
        A position vector of the observed object
    
        uvw: numpy array
        
        An instantaneous velocity vector of the observed object
        """
        
        obj = Horizons(id = name, location = loc, epochs = t, id_type = 'majorbody')
        obj1 = obj.vectors(aberrations = cor, refplane = 'earth')
        xyz = np.array([float(obj1['x']),float(obj1['y']),float(obj1['z'])])
        uvw = np.array([float(obj1['vx']),float(obj1['vy']),float(obj1['vz'])])
        obj2 = obj.ephemerides(refsystem = 'J2000', extra_precision = True)
        radec = np.array([float(obj2['RA']),float(obj2['DEC']),float(obj1['range'])])
        return xyz,uvw,radec
    return get_target_xyz 

In [8]:
def CordConv(xyz):
    '''
    This function takes in a position vector of a body relative to an observer and returns a radec.
    
    Arguments:
    
    xyz: numpy array
    
    Should be three values, the x,y,z of the position vector
    
    Returns:
    
    radec: numpy array
    
    Two values, the first for right ascension and the second for declination
    
    '''
    DEC = -(np.arccos(xyz[2]/np.linalg.norm(xyz))-np.pi/2)
    RA = (np.arctan2(xyz[1],xyz[0]))
    while (RA > 2*np.pi):
        RA -= 2*np.pi   
    while (RA < 0):
        RA += 2*np.pi   
    return np.array([RA*180/np.pi,DEC*180/np.pi])

In [38]:
xyzs = []
get_earth = get_astroquery_function('0','geometric','f51')
for j in range(0,len(times)):
    thor = Time(times[j], format='jd', scale='utc')
    thor = thor.tt.value
    F51Sun = get_earth(thor)
    xyzs.append(F51Sun)
    print(j)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [65]:
def Integrator2(t,bsolar,msolar,iJovian,body1,body2,Ms):
    '''
    Integrates the system. If body1 or body2 refer to the Jupiter Barycenter, while there are bodies in the Jovian system
    teh function will compute the barycenter in the simulation
    
    Arguments:
    
    bsolar
    msolar
    iJovian
    
    body1 and body2: name in str
    
    The spice names of the bodies that we want to look into to check the error of our integrator
    
     '''
    sim = SimStart3(t,bsolar,msolar,iJovian,Ms)
    year = 365.25 # days
    tmax = 1*year
    sim.integrate(tmax)
    return sim

In [70]:
def Integrator(t,ten,bsolar,msolar,iJovian,Ms,arr):
    '''
    Integrates the system. If body1 or body2 refer to the Jupiter Barycenter, while there are bodies in the Jovian system
    teh function will compute the barycenter in the simulation
    
    Arguments:
    
    bsolar
    msolar
    iJovian
    
    body1 and body2: name in str
    
    The spice names of the bodies that we want to look into to check the error of our integrator
    
     '''
    sim = SimStart2(t,bsolar,msolar,iJovian,Ms,arr)
    year = 365.25 # days
    tmax = (ten - t)/86400 
    sim.integrate(tmax)
    return sim

In [9]:
def FinderSpicef51(name,t):
    '''
    
    Finds the radec of objects in the jovian system from the pansstars observatory. Used to test the correspondence
    of spice and Horizons.
    
    Arguments:
    
    name: str
    
    The name of the object in spice documentation. See Summary_Names.txt
    
    t: float
    
    A julian date
    
    '''
    thor = Time(t, format='jd', scale='utc')
    thor = thor.tt.value
    tstr = "jd " + str(t) + " utc"
    ts = spice.str2et(tstr)
    get_earth = get_astroquery_function('0','geometric','f51')
    get_jup = get_spice_function(name,"NONE","0")
    exyz = get_earth(thor)
    jxyz = get_jup(ts)
    fxyz = LT(ts, -exyz[0], get_jup)
    vectors = fxyz[0] + exyz[0]
    return CordConv(vectors)

In [10]:
def FinderHorizonsf51(name,t):
    '''
    
    Finds the radec of objects in the jovian system from the pansstars observatory from the Horizons data base.
    
    Arguments:
    
    name: str
    
    The name of the object in spice documentation. See Summary_Names.txt
    
    t: float
    
    A julian date
    
    '''
    thor = Time(t, format='jd', scale='utc')
    thor = thor.tt.value
    get_pans = get_astroquery_function(name,'astrometric','f51')
    vecs2 = get_pans(thor)
    return CordConv(vecs2[0])

In [64]:
def SimStart3(t,bsolar,msolar,iJovian,Ms):
    '''
    Starts the simulation. Adds all particles
    
    Arguments:
    
    bsolar: list of strs
    
    The names of the bodies added. Typically: '1','2','3','4', 'Jupiter','Io','Ananke'. 'Jupiter' must be added
    last followed by the satellites in the system
    
    msolar: list of floats
    
    The masses of the bodies in bsolar. Must be at the order of bsolar
    
    iJovian: list, int
    
    The indexes (+1) that the satellites in the solar system have in bsolar
    
    '''
    
    sim = rebound.Simulation()
    k = np.sqrt(Ms*10**9)*(1.49597870700*10**11)**(-1.5) *86400.00000
    sim.G = k**2
    sim.add(m=1.)
    for i in range(0,len(bsolar)):
        get_planet = get_spice_function(bsolar[i],'NONE','SUN')
        xyz,uvw,radec = get_planet(t)
        xyz = xyz
        #uvw = (uvw*24*3600)
        sim.add(m=msolar[i],x=xyz[0],y=xyz[1],z=xyz[2],vx=uvw[0],vy=uvw[1],vz=uvw[2])
    ps = sim.particles
    for j in range(1,len(ps)):
        if j not in iJovian:
            ps[j].calculate_orbit(primary=ps[0])
        elif bsolar[j-1] == 'Moon':
            ps[j].calculate_orbit(primary=ps[bsolar.index('Earth')+1])
        else:
            if 'Jupiter' in bsolar:
                ps[j].calculate_orbit(primary=ps[bsolar.index('Jupiter')+1])
            else:
                ps[j].calculate_orbit(primary=ps[bsolar.index('5')+1])
    sim.move_to_com()
    return sim

In [11]:
def SimStart2(t,bsolar,msolar,iJovian,Ms,arr):
    '''
    Starts the simulation. Adds all particles
    
    Arguments:
    
    bsolar: list of strs
    
    The names of the bodies added. Typically: '1','2','3','4', 'Jupiter','Io','Ananke'. 'Jupiter' must be added
    last followed by the satellites in the system
    
    msolar: list of floats
    
    The masses of the bodies in bsolar. Must be at the order of bsolar
    
    iJovian: list, int
    
    The indexes (+1) that the satellites in the solar system have in bsolar
    
    '''
    
    sim = rebound.Simulation()
    k = np.sqrt(Ms*10**9)*(1.49597870700*10**11)**(-1.5) *86400.00000
    sim.G = k**2
    sim.add(m=1.)
    for i in range(0,len(bsolar)):
        get_planet = get_spice_function(bsolar[i],'NONE','SUN')
        xyz,uvw,radec = get_planet(t)
        xyz = xyz
        #uvw = (uvw*24*3600)
        sim.add(m=msolar[i],x=xyz[0],y=xyz[1],z=xyz[2],vx=uvw[0],vy=uvw[1],vz=uvw[2])
    ps = sim.particles
    len(ps)
    for i in range(0,len(arr)):
        sim.add(a=arr[i][0],e=arr[i][1],inc=arr[i][2],Omega=arr[i][3],omega=arr[i][4],f=arr[i][5],primary=ps[bsolar.index('Jupiter')+1])  
    sim.n_active = len(bsolar)+1
    sim.move_to_com()
    return sim

In [36]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
df = pd.read_excel('/Users/user/wfilter.xlsx', sheetname='Sheet1')
print("Column headings:")
print(df.columns)
IDs = np.array(df['ID'])
times = np.array(df['JDUTC'])

Column headings:
Index(['ID', 'smf-name', 'smf-filepath', 'reduction', 'JDUTC', 'RA', 'Dec',
       'filter', 'exp-time', 'UVx', 'UVy', 'UVz', 'obseqx', 'obseqy', 'obseqz',
       'obseclx', 'obseclsy', 'obseclz'],
      dtype='object')


/anaconda3/lib/python3.6/site-packages/pandas/io/excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


In [37]:
def Seeder(n):
    arr = []
    for i in range(0,n):
        e = random.uniform(0,1)
        a = random.uniform(0.05,(0.177/(1+e)))
        i = random.uniform(0,np.pi)
        Om = random.uniform(0,2*np.pi)
        om = random.uniform(0,2*np.pi)
        k = random.uniform(0,2*np.pi)
        arr.append(np.array([a,e,i,Om,om,k]))
    return arr

In [63]:
def FinderReboundf51(t,years,arr,times,vecs,IDs):
    '''
    
    Finds the radec of objects in the jovian system from the pansstars observatory a number of years before
    the time of observation and integrates it to the time of observation using rebound.
    
    Arguments:
    
    name: str
    
    The name of the object in spice documentation. See Summary_Names.txt
    
    t: float
    
    A julian date
    
    '''
    tstr = "jd " + str(t) + " utc"
    ts = spice.str2et(tstr)
    get_earth = get_astroquery_function('0','geometric','f51')
    sec_year = 86400*365.25
    sim = Integrator(ts - years*sec_year,ts,['1','2','3','4','6','7','8',
        'Jupiter','Io','Europa','Ganymede','Callisto'],[MMercuryB,MVenusB,
        MEarthB,MMarsB,MSaturnB,MUranusB,MNeptuneB,MJupiter,MIo,MEuropa,MGanymede,
        MCallisto],[9,10,11,12,13,14,15],GMSun,arr)
    ps = sim.particles
    radec = []
    ttemp = ts
    for j in range(0,len(times)):
        temp = "jd " + str(times[j]) + " utc"
        ttemp2 = spice.str2et(temp)
        sim.integrate(sim.t + (ttemp2-ttemp)/86400)
        ps = sim.particles
        ttemp = ttemp2
        F51 = vecs[j][0]
        for i in range(13,len(ps)):
            dist = np.sqrt((ps[i].x-ps[3].x)**2 + (ps[i].y-ps[3].y)**2 + (ps[i].z-ps[3].z)**2)
            ltime = float(((dist*u.AU).to(u.m)/c)/u.s)
            sim.integrate(sim.t - ltime/(24*3600))
            ps = sim.particles
            AnSun = np.array([float(ps[i].x),float(ps[i].y),float(ps[i].z)])
            vectors = AnSun + F51Sun[0]
            radec.append(np.array([CordConv(vectors),times[j],IDs[j]]))
            sim.integrate(sim.t + ltime/(24*3600))
            ps = sim.particles
    return radec

In [284]:
def FinderReboundf51_2(t,years,arr,times,vecs,IDs):
    '''
    
    Finds the radec of objects in the jovian system from the pansstars observatory a number of years before
    the time of observation and integrates it to the time of observation using rebound.
    
    Arguments:
    
    name: str
    
    The name of the object in spice documentation. See Summary_Names.txt
    
    t: float
    
    A julian date
    
    '''
    thor = Time(t, format='jd', scale='utc')
    thor = thor.tt.value
    tstr = "jd " + str(t) + " utc"
    ts = spice.str2et(tstr)
    get_earth = get_astroquery_function('0','geometric','f51')
    sec_year = 86400*365.25
    sim = Integrator(ts - years*sec_year,ts,['1','2','3','4','6','7','8',
        'Jupiter','Io','Europa','Ganymede','Callisto'],[MMercuryB,MVenusB,
        MEarthB,MMarsB,MSaturnB,MUranusB,MNeptuneB,MJupiter,MIo,MEuropa,MGanymede,
        MCallisto],[9,10,11,12,13,14,15],GMSun,arr)
    ttemp = ts
    radec = []
    print(sim.t)
    for j in range(0,len(times)):
        temp = "jd " + str(times[j]) + " utc"
        ttemp2 = spice.str2et(temp)
        cqtime.append(sim.t)
        k = sim.t + (ttemp2-ttemp)/86400
        ttemp = ttemp2
        sim.integrate(k)
        thor = Time(times[j], format='jd', scale='utc')
        thor = thor.tt.value
        ps = sim.particles
        dist = np.sqrt((ps[-1].x-ps[3].x)**2 + (ps[-1].y-ps[3].y)**2 + (ps[-1].z-ps[3].z)**2)
        ltime = float(((dist*u.AU).to(u.m)/c)/u.s)
        k = sim.t - ltime/(24*3600)
        sim.integrate(k)
        ps = sim.particles
        AnSun = np.array([float(ps[-1].x),float(ps[-1].y),float(ps[-1].z)])
        #F51Sun = get_earth(thor)
        #vecs.append(F51Sun[0])
        vectors = AnSun + vecs[j][0]
        radec.append(np.array([CordConv(vectors),times[j],IDs[j]]))
        k = sim.t + ltime/(24*3600)
        sim.integrate(k)
        print(j)
    return radec

In [285]:
radec3 = FinderReboundf51_2(times[0]-1,1,arr,times,xyzs,IDs)

365.25
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275

In [291]:
radk = []
for i in range(0,len(radec2)):
    radk.append(radec2[i])

In [295]:
(np.array(radk)-np.array(radec2))[50]

array([0., 0.])

In [283]:
radec2[0]

array([357.93810069,  -2.34655375])

In [286]:
%%time
radecmain = [] 
arrsmain = []
for i in range(0,100):
    arr = Seeder(1)
    print(arr)
    radecmain.append(FinderReboundf51((times[0]-1),1,arr,times,xyzs,IDs))
    arrsmain.append(arr)
    print(i)
df = pd.DataFrame(arrsmain)
filepath = '/Users/user/seedermain.csv'
df.to_csv(filepath, index=False)


[array([0.10629572, 0.53271944, 1.60527843, 4.91193792, 0.54682524,
       4.29520069])]
0
[array([0.10381869, 0.38327965, 2.39533277, 1.45398514, 1.28695268,
       4.77299913])]
1
[array([0.09426228, 0.86557651, 2.89695626, 6.25669082, 2.35868623,
       3.41370251])]
2
[array([0.05191836, 0.10502595, 1.26086655, 3.26985029, 3.5005983 ,
       1.27888531])]
3
[array([0.08774757, 0.46437072, 2.51073665, 5.93705445, 4.30960779,
       1.63006453])]
4
[array([0.05819702, 0.06767975, 1.62965318, 1.50455043, 2.03340318,
       1.81477105])]
5
[array([0.06121061, 0.64502528, 0.99594947, 4.06012061, 2.16475098,
       3.87727387])]
6
[array([0.07759012, 0.61635051, 1.62089889, 4.78827623, 2.90228689,
       1.47892469])]
7
[array([0.09190882, 0.89084992, 2.36196654, 2.22312466, 1.89230059,
       0.1648781 ])]
8
[array([0.05082287, 0.33078207, 0.95256434, 3.20217641, 1.35687765,
       5.04532381])]
9
[array([0.07687133, 0.32506155, 2.3386527 , 0.19314814, 1.28968539,
       2.32289382])]
1

88
[array([0.0674432 , 0.77491239, 2.12678013, 5.49381682, 3.62321314,
       3.57802526])]
89
[array([0.12017565, 0.43000741, 2.740662  , 0.23419495, 2.15052032,
       6.23041549])]
90
[array([0.08678479, 0.64429316, 0.56590691, 2.90961351, 0.30353256,
       2.05809572])]
91
[array([0.14602262, 0.04984955, 2.62835169, 5.84839947, 3.6467372 ,
       0.97765346])]
92
[array([0.09814103, 0.43336677, 1.19294795, 0.0094252 , 3.60459027,
       3.42199556])]
93
[array([0.06116393, 0.69624555, 0.12308909, 3.62283414, 2.9314717 ,
       5.44912804])]
94
[array([0.07423638, 0.53758543, 3.04307499, 3.90334842, 6.1491087 ,
       2.60828169])]
95
[array([0.07290978, 0.54677946, 1.56116938, 0.09494311, 3.66444139,
       5.20245689])]
96
[array([0.05931838, 0.72392343, 1.54770584, 0.89127364, 3.82664425,
       3.32368778])]
97
[array([0.06733028, 0.65196277, 0.09354333, 2.07688733, 5.75286878,
       2.76181237])]
98
[array([0.08200235, 0.7419746 , 1.63465884, 4.83979003, 1.00210086,
       5.

In [287]:
boolsbeg = []
boolsend = []
for i in range(0,99):
    get_earth = get_astroquery_function('5','astrometric','f51')
    thor = Time(times[50], format='jd', scale='utc')
    thor = thor.tt.value
    F51 = get_earth(thor)
    diffra = abs(radecmain[i][50][0][0]-F51[2][0])
    diffdec = abs(radecmain[i][50][0][1]-F51[2][1])
    Hill = (0.177/np.linalg.norm(F51[0])) * 180/np.pi
    if (diffra < Hill) and (diffdec<Hill):
        boolsbeg.append(True)
    else:
        boolsbeg.append(False)
    thor = Time(times[70], format='jd', scale='utc')
    thor = thor.tt.value
    F51 = get_earth(thor)
    diffra = abs(radecmain[i][70][0][0]-F51[2][0])
    diffdec = abs(radecmain[i][70][0][1]-F51[2][1])
    Hill = (0.177/np.linalg.norm(F51[0])) * 180/np.pi
    if (diffra < Hill) and (diffdec<Hill):
        boolsend.append(True)
    else:
        boolsend.append(False)
    

In [288]:
boolsbeg.count(True),boolsend.count(True)

(0, 0)